# RFM Analysis

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
# Import the libraries
import pandas as pd
import datetime as dt
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
# import the dataset
online_retail=pd.read_csv('/content/drive/MyDrive/Python Datasets/rfm Analysis data.csv',encoding='unicode_escape')
df=online_retail.copy()
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom


In [ ]:
df.shape

(525461, 8)

In [4]:
df.isnull().sum()

Invoice             0
StockCode           0
Description      2928
Quantity            0
InvoiceDate         0
Price               0
Customer ID    107927
Country             0
dtype: int64

In [5]:
df.dropna(inplace=True) # drop null values

In [6]:
# How many unique items available in dataset?
df['Description'].nunique()

4459

In [7]:
# How many product in the dataset?
df['Description'].value_counts()

Description
WHITE HANGING HEART T-LIGHT HOLDER    3245
REGENCY CAKESTAND 3 TIER              1872
STRAWBERRY CERAMIC TRINKET BOX        1536
ASSORTED COLOUR BIRD ORNAMENT         1376
HOME BUILDING BLOCK WORD              1229
                                      ... 
BLUE OWL DECORATION                      1
PINK OWL DECORATION                      1
IVORY WALL CLOCK                         1
RED WALL CLOCK                           1
BAKING MOULD EASTER EGG MILK CHOC        1
Name: count, Length: 4459, dtype: int64

In [8]:
#Rank the most 5 ordered products form most to least
df.groupby("Description").agg({"Quantity":"sum"}).sort_values(by="Quantity",ascending=False).head(5)

,Quantity
Description,
WHITE HANGING HEART T-LIGHT HOLDER,55861
WORLD WAR 2 GLIDERS ASSTD DESIGNS,54274
BROCADE RING PURSE,47430
PACK OF 72 RETRO SPOT CAKE CASES,44507
ASSORTED COLOUR BIRD ORNAMENT,44120


In [9]:
df['Invoice'].value_counts() # calculate the count of every invoices

Invoice
500356     270
511522     255
531382     251
507235     250
511051     248
          ... 
C507230      1
494888       1
494892       1
494899       1
C524997      1
Name: count, Length: 23587, dtype: int64

In [10]:
df=df[~df['Invoice'].str.contains('C',na=False)] # Removes the values that contains "C" from Invoice column

In [11]:
#calculate total revenue
df['TotalPrice']=df['Quantity']*df['Price']
df.head()

,Invoice,StockCode,Description,Quantity,InvoiceDate,Price,Customer ID,Country,TotalPrice
0,489434,85048,15CM CHRISTMAS GLASS BALL 20 LIGHTS,12,12/1/2009 7:45,6.95,13085.0,United Kingdom,83.4
1,489434,79323P,PINK CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom,81.0
2,489434,79323W,WHITE CHERRY LIGHTS,12,12/1/2009 7:45,6.75,13085.0,United Kingdom,81.0
3,489434,22041,"RECORD FRAME 7"" SINGLE SIZE",48,12/1/2009 7:45,2.10,13085.0,United Kingdom,100.8
4,489434,21232,STRAWBERRY CERAMIC TRINKET BOX,24,12/1/2009 7:45,1.25,13085.0,United Kingdom,30.0


In [13]:
df.dtypes

Invoice                object
StockCode              object
Description            object
Quantity                int64
InvoiceDate    datetime64[ns]
Price                 float64
Customer ID           float64
Country                object
TotalPrice            float64
dtype: object

In [12]:
df['InvoiceDate']=pd.to_datetime(df['InvoiceDate']) # Convert InvoiceDate from object to datatime datatype

In [14]:
df["InvoiceDate"].max() # get the most recent date of customer visit

Timestamp('2010-12-09 20:01:00')

In [15]:
# Determine the analysis date for the recency, using aggregation create Recency, Frequency and Monetry values for each customerID
today_date=dt.datetime(2010,12,10)
rfm= df.groupby('Customer ID').agg({'InvoiceDate':lambda date:(today_date-date.max()).days,
                                    'Invoice':lambda num:num.nunique() ,
                                    'TotalPrice':lambda TotalPrice: TotalPrice.sum()})
rfm.columns=['Recency','Frequency','Monetary']


In [16]:
rfm

,Recency,Frequency,Monetary
Customer ID,,,
12346.0,164,11,372.86
12347.0,2,2,1323.32
12348.0,73,1,222.16
12349.0,42,3,2671.14
12351.0,10,1,300.93
...,...,...,...
18283.0,17,6,641.77
18284.0,66,1,461.68
18285.0,295,1,427.00


In [17]:
rfm=rfm[rfm['Monetary']>0] # Filter the data according to values of Monetry more than 0
rfm.describe()

,Recency,Frequency,Monetary
count,4312.000000,4312.000000,4312.000000
mean,90.172542,4.455705,2048.238236
std,96.861457,8.170213,8914.481280
min,0.000000,1.000000,2.950000
25%,17.000000,1.000000,307.987500
50%,52.000000,2.000000,706.020000
75%,135.000000,5.000000,1723.142500
max,373.000000,205.000000,349164.350000


In [18]:
import warnings
warnings.filterwarnings('ignore')

# Calculate the scores for Recency, Monetry and Frequency

In [19]:
rfm["recency_score"]=pd.qcut(rfm['Recency'],5,labels=[5,4,3,2,1])

In [20]:
rfm["frequency_score"]=pd.qcut(rfm['Frequency'].rank(method='first'),5,labels=[1,2,3,4,5])
rfm["monetary_score"]=pd.qcut(rfm['Monetary'],5,labels=[1,2,3,4,5])

In [21]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score
Customer ID,,,,,,
12346.0,164,11,372.86,2,5,2
12347.0,2,2,1323.32,5,2,4
12348.0,73,1,222.16,2,1,1
12349.0,42,3,2671.14,3,3,5
12351.0,10,1,300.93,5,1,2


In [22]:
rfm["RFM_SCORE"]=rfm['recency_score'].astype(str)+rfm['frequency_score'].astype(str)+rfm['monetary_score'].astype(str)

In [23]:
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE
Customer ID,,,,,,,
12346.0,164,11,372.86,2,5,2,252
12347.0,2,2,1323.32,5,2,4,524
12348.0,73,1,222.16,2,1,1,211
12349.0,42,3,2671.14,3,3,5,335
12351.0,10,1,300.93,5,1,2,512


In [24]:
# create the different segments of the customers
seg_map={
    r'[1-2][1-2]':'hibernating',
    r'[1-2][3-4]':'at_Risk',
    r'[1-2]5':'cant_loose',
    r'3[1-2]':'about_to_sleep',
    r'33':'need_attention',
    r'[3-4][4-5]':'loyal_customers',
    r'41':'promising',
    r'51':'new_customers',
    r'[4-5][2-3]':'potential_loyalists',
    r'5[4-5]':'champions'
}
rfm['segment']=rfm['RFM_SCORE'].replace(seg_map,regex=True)
rfm.head()

,Recency,Frequency,Monetary,recency_score,frequency_score,monetary_score,RFM_SCORE,segment
Customer ID,,,,,,,,
12346.0,164,11,372.86,2,5,2,252,cant_loose2
12347.0,2,2,1323.32,5,2,4,524,5at_Risk
12348.0,73,1,222.16,2,1,1,211,hibernating1
12349.0,42,3,2671.14,3,3,5,335,need_attention5
12351.0,10,1,300.93,5,1,2,512,5hibernating


In [30]:
rfm[["segment","Recency","Frequency","Monetary"]].groupby("segment").agg(["mean","count"]).round()

Recency       Frequency       Monetary      
                        mean count      mean count     mean count
segment                                                          
3about_to_sleep         53.0    58       2.0    58    356.0    58
3at_Risk                55.0    93       1.0    93    847.0    93
3cant_loose             61.0     3       2.0     3   3064.0     3
3hibernating            52.0   247       1.0   247    257.0   247
4about_to_sleep         24.0    55       2.0    55    327.0    55
4at_Risk                25.0    66       2.0    66    917.0    66
4cant_loose             31.0     4       2.0     4   5482.0     4
4hibernating            24.0   176       1.0   176    262.0   176
4loyal_customers        24.0    47       3.0    47   1714.0    47
4need_attention         24.0    63       2.0    63    696.0    63
5about_to_sleep          7.0    40       2.0    40    347.0    40
5at_Risk                 8.0    26       2.0    26    967.0    26
5cant_loose             10.0     3       2.0     3   2635.0     3
5hibernating             8.0    92       1.0    92    250.0    92
5loyal_customers         7.0   161       4.0   161   1956.0   161
5need_attention          6.0    50       3.0    50    716.0    50
5potential_loyalists     7.0    87       4.0    87    702.0    87
5promising               7.0     3       3.0     3    177.0     3
at_Risk1               187.0    52       2.0    52    167.0    52
at_Risk2               174.0   122       3.0   122    371.0   122
at_Risk3               147.0   208       3.0   208    723.0   208
at_Risk4               139.0   163       4.0   163   1441.0   163
at_Risk5               124.0    66       4.0    66   4352.0    66
cant_loose1            297.0     1       8.0     1    181.0     1
cant_loose2            143.0     2       9.0     2    331.0     2
cant_loose3             99.0     7       7.0     7    780.0     7
cant_loose4            116.0    26       8.0    26   1731.0    26
cant_loose5            126.0    41      10.0    41   6448.0    41
champions4               6.0    94       9.0    94   1595.0    94
champions5               5.0   350      19.0   350  11624.0   350
hibernating1           222.0   509       1.0   509    146.0   509
hibernating2           215.0   324       1.0   324    353.0   324
hibernating3           177.0   125       1.0   125    692.0   125
hibernating4           200.0    42       1.0    42   1389.0    42
hibernating5           186.0    15       1.0    15   5087.0    15
loyal_customers1        56.0     1       9.0     1     97.0     1
loyal_customers2        37.0    25       4.0    25    419.0    25
loyal_customers3        36.0   129       4.0   129    767.0   129
loyal_customers4        37.0   275       5.0   275   1502.0   275
loyal_customers5        34.0   312      10.0   312   4856.0   312
need_attention3         52.0    78       2.0    78    697.0    78
need_attention4         52.0    49       3.0    49   1460.0    49
need_attention5         53.0    22       3.0    22   3313.0    22